In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
import datetime

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
esg = pd.read_csv('ESG_scoredata/ESG_scoredata.csv')

In [4]:
ratios = pd.read_csv('ESG_scoredata/financial_ratios.csv')

In [5]:
securitydaily = pd.read_csv('ESG_scoredata/security_daily.csv')

/var/folders/mc/zs1s69ln6dlfsp0q79s_czc40000gn/T/ipykernel_51042/2359471464.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  securitydaily = pd.read_csv('ESG_scoredata/security_daily.csv')


**Market Capitalization Calculation (annual average for each company)**

In [6]:
securitydaily.head()

,gvkey,iid,datadate,tic,cusip,conm,ajexdi,cshoc,prccd,trfd
0,1003,01,04/03/2014,ANTQ,000354100,A.A. IMPORTING CO INC,1.0,2683000.0,0.02,NaN
1,1003,01,04/04/2014,ANTQ,000354100,A.A. IMPORTING CO INC,1.0,2683000.0,0.02,NaN
2,1003,01,04/07/2014,ANTQ,000354100,A.A. IMPORTING CO INC,1.0,2683000.0,0.02,NaN
3,1003,01,04/08/2014,ANTQ,000354100,A.A. IMPORTING CO INC,1.0,2683000.0,0.02,NaN
4,1003,01,04/09/2014,ANTQ,000354100,A.A. IMPORTING CO INC,1.0,2683000.0,0.02,NaN


In [7]:
securitydaily.isna().sum()

gvkey              0
iid                0
datadate           0
tic             4933
cusip              0
conm               0
ajexdi         20099
cshoc        1031557
prccd          21382
trfd        19640760
dtype: int64

In [8]:
securitydaily.dropna(subset=['ajexdi','cshoc','prccd','tic'],inplace =True)

In [9]:
securitydaily['mktcap']= securitydaily['prccd']*securitydaily['cshoc']/securitydaily['ajexdi']

In [10]:
securitydaily['datadate'] = pd.to_datetime(securitydaily['datadate'])
securitydaily['year'] = securitydaily['datadate'].dt.year

In [11]:
securitydaily['mktcap_annual'] = securitydaily.groupby(['gvkey','tic','year'])['mktcap'].transform('mean')

In [12]:
securitydaily.tail()

,gvkey,iid,datadate,tic,cusip,conm,ajexdi,cshoc,prccd,trfd,mktcap,year,mktcap_annual
46931547,353945,90,2022-12-23,ACLLY,00449R109,ACCELLERON INDUSTRIES AG,1.0,94500000.0,20.483,NaN,1.935644e+09,2022,1.747278e+09
46931548,353945,90,2022-12-27,ACLLY,00449R109,ACCELLERON INDUSTRIES AG,1.0,94500000.0,20.641,NaN,1.950574e+09,2022,1.747278e+09
46931549,353945,90,2022-12-28,ACLLY,00449R109,ACCELLERON INDUSTRIES AG,1.0,94500000.0,20.401,NaN,1.927894e+09,2022,1.747278e+09
46931550,353945,90,2022-12-29,ACLLY,00449R109,ACCELLERON INDUSTRIES AG,1.0,94500000.0,20.720,NaN,1.958040e+09,2022,1.747278e+09
46931551,353945,90,2022-12-30,ACLLY,00449R109,ACCELLERON INDUSTRIES AG,1.0,94500000.0,20.693,NaN,1.955489e+09,2022,1.747278e+09


In [13]:
mktcap_annualdata = securitydaily.drop(columns=['iid', 'datadate','ajexdi','cshoc','prccd','trfd','mktcap','cusip']).groupby(['gvkey','tic','year']).first().reset_index()

In [14]:
mktcap_annualdata.head()

,gvkey,tic,year,conm,mktcap_annual
0,1003,ANTQ,2014,A.A. IMPORTING CO INC,5.366000e+04
1,1003,ANTQ,2015,A.A. IMPORTING CO INC,5.366000e+04
2,1003,ANTQ,2016,A.A. IMPORTING CO INC,5.366000e+04
3,1004,AIR,2013,AAR CORP,9.202797e+08
4,1004,AIR,2014,AAR CORP,1.059264e+09


**Financial Ratio Data Cleaning**

*(created annual data, 'year' corresponds to the year in 'adate', take the first observation in the same 'adate' group as a representative for each company-year level)*

In [15]:
ratios.loc[ratios['gvkey']==24447].head()

,gvkey,adate,qdate,public_date,bm,gpm,roe,debt_assets,curr_ratio,at_turn,divyield,TICKER,cusip
177155,24447,12/31/2011,09/30/2012,01/31/2013,0.738,0.951,0.120,0.889,NaN,0.055,2.71%,BOKF,05561Q20
177156,24447,12/31/2012,12/31/2012,02/28/2013,0.795,0.955,0.123,0.894,NaN,0.054,2.56%,BOKF,05561Q20
177157,24447,12/31/2012,12/31/2012,03/31/2013,0.795,0.955,0.123,0.894,NaN,0.054,2.44%,BOKF,05561Q20
177158,24447,12/31/2012,12/31/2012,04/30/2013,0.795,0.955,0.123,0.894,NaN,0.054,2.43%,BOKF,05561Q20
177159,24447,12/31/2012,03/31/2013,05/31/2013,0.704,0.965,0.122,0.889,NaN,0.054,2.33%,BOKF,05561Q20


In [16]:
ratios_cleaned = ratios.drop(columns=['qdate', 'public_date','cusip']).groupby(['adate','gvkey']).first().reset_index()
ratios_cleaned['adate'] = pd.to_datetime(ratios_cleaned['adate'])
ratios_cleaned['year'] = ratios_cleaned['adate'].dt.year
ratios_cleaned = ratios_cleaned.drop(columns=['adate','divyield'])

In [17]:
ratios_cleaned.loc[ratios_cleaned['gvkey'] == 24447].head()

,gvkey,bm,gpm,roe,debt_assets,curr_ratio,at_turn,TICKER,year
10042,24447,0.738,0.951,0.120,0.889,NaN,0.055,BOKF,2011
12807,24447,0.795,0.955,0.123,0.894,NaN,0.054,BOKF,2012
15813,24447,0.661,0.968,0.106,0.887,NaN,0.049,BOKF,2013
18835,24447,0.798,0.950,0.092,0.885,NaN,0.048,BOKF,2014
21786,24447,0.820,0.932,0.088,0.896,NaN,0.047,BOKF,2015


In [18]:
ratios_cleaned.isna().sum()

gvkey             0
bm             1902
gpm            2386
roe            1990
debt_assets       0
curr_ratio     7397
at_turn        2129
TICKER          443
year              0
dtype: int64

In [19]:
ratios_cleaned.shape

(41765, 9)

In [20]:
#merging esg dataset and financial ratios dataset
esg_ratio = pd.merge(esg, ratios_cleaned, left_on=['gvkey','assessmentyear','ticker'],right_on=['gvkey','year','TICKER'], how='inner')

In [21]:
esg_ratio.isna().sum()

scoreid                              0
dimensionscoreid                     0
institutionid                        0
assessmentyear                       0
scoredate                            0
csaindustryname                      0
csaindustrygroupname                 0
csaindustryclassificationname        0
csasectorname                        0
scoretype                            0
dimensionname                        0
aspectname                           0
scorevalue                        6693
scoreweight                      18531
industryrank                      6693
companyid                            0
gvkey                                0
ticker                               0
companyname                          0
country                              0
simpleindustry                       0
bm                               18334
gpm                               3164
roe                              15904
debt_assets                          0
curr_ratio               

In [22]:
#esg_ratio[esg_ratio['scorevalue'].isna()]

In [23]:
#droping observations without scorevalue data
esg_ratio.dropna(subset=['scorevalue'],inplace =True)

In [24]:
esg_ratio.shape

(486177, 29)

In [25]:
esg_ratio.drop(columns=['scoreid','dimensionscoreid','institutionid','assessmentyear','TICKER'],inplace=True)

In [26]:
esg_ratio.head()

,scoredate,csaindustryname,csaindustrygroupname,csaindustryclassificationname,csasectorname,scoretype,dimensionname,aspectname,scorevalue,scoreweight,industryrank,companyid,gvkey,ticker,companyname,country,simpleindustry,bm,gpm,roe,debt_assets,curr_ratio,at_turn,year
0,04/01/2020,BNK Banks,Banks,SAM GICS,Financials,S&P Global ESG Score,S&P Global ESG Score,S&P Global ESG Score,26.0,NaN,241.0,323261,24447,BOKF,BOK Financial Corporation,United States,Banks,0.791,0.792,0.108,0.885,NaN,0.056,2019
1,04/01/2020,BNK Banks,Banks,SAM GICS,Financials,Criteria,Environmental Dimension,Climate Strategy,42.0,6.0,198.0,323261,24447,BOKF,BOK Financial Corporation,United States,Banks,0.791,0.792,0.108,0.885,NaN,0.056,2019
2,04/01/2020,BNK Banks,Banks,SAM GICS,Financials,Criteria,Economic Governance Dimension,Sustainable Finance,12.0,9.0,264.0,323261,24447,BOKF,BOK Financial Corporation,United States,Banks,0.791,0.792,0.108,0.885,NaN,0.056,2019
3,04/01/2020,BNK Banks,Banks,SAM GICS,Financials,Criteria,Social Dimension,Social Reporting,0.0,4.0,302.0,323261,24447,BOKF,BOK Financial Corporation,United States,Banks,0.791,0.792,0.108,0.885,NaN,0.056,2019
4,04/01/2020,BNK Banks,Banks,SAM GICS,Financials,Criteria,Economic Governance Dimension,Privacy Protection,22.0,2.0,254.0,323261,24447,BOKF,BOK Financial Corporation,United States,Banks,0.791,0.792,0.108,0.885,NaN,0.056,2019


In [27]:
#combining the mktcap data to the esg data and financial ratio data
combined_df = pd.merge(esg_ratio, mktcap_annualdata, left_on=['gvkey','year','ticker'],right_on=['gvkey','year','tic'], how='inner')

In [28]:
combined_df.drop(columns=['tic'],inplace=True)

In [29]:
combined_df

,scoredate,csaindustryname,csaindustrygroupname,csaindustryclassificationname,csasectorname,scoretype,dimensionname,aspectname,scorevalue,scoreweight,industryrank,companyid,gvkey,ticker,companyname,country,simpleindustry,bm,gpm,roe,debt_assets,curr_ratio,at_turn,year,conm,mktcap_annual
0,04/01/2020,BNK Banks,Banks,SAM GICS,Financials,S&P Global ESG Score,S&P Global ESG Score,S&P Global ESG Score,26.0,NaN,241.0,323261,24447,BOKF,BOK Financial Corporation,United States,Banks,0.791,0.792,0.108,0.885,NaN,0.056,2019,BOK FINANCIAL CORP,5.807121e+09
1,04/01/2020,BNK Banks,Banks,SAM GICS,Financials,Criteria,Environmental Dimension,Climate Strategy,42.0,6.0,198.0,323261,24447,BOKF,BOK Financial Corporation,United States,Banks,0.791,0.792,0.108,0.885,NaN,0.056,2019,BOK FINANCIAL CORP,5.807121e+09
2,04/01/2020,BNK Banks,Banks,SAM GICS,Financials,Criteria,Economic Governance Dimension,Sustainable Finance,12.0,9.0,264.0,323261,24447,BOKF,BOK Financial Corporation,United States,Banks,0.791,0.792,0.108,0.885,NaN,0.056,2019,BOK FINANCIAL CORP,5.807121e+09
3,04/01/2020,BNK Banks,Banks,SAM GICS,Financials,Criteria,Social Dimension,Social Reporting,0.0,4.0,302.0,323261,24447,BOKF,BOK Financial Corporation,United States,Banks,0.791,0.792,0.108,0.885,NaN,0.056,2019,BOK FINANCIAL CORP,5.807121e+09
4,04/01/2020,BNK Banks,Banks,SAM GICS,Financials,Criteria,Economic Governance Dimension,Privacy Protection,22.0,2.0,254.0,323261,24447,BOKF,BOK Financial Corporation,United States,Banks,0.791,0.792,0.108,0.885,NaN,0.056,2019,BOK FINANCIAL CORP,5.807121e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482263,03/18/2022,TSV IT services,Software & Services,SAM GICS,Information Technology,Criteria,Economic Governance Dimension,Business Ethics,59.0,8.0,25.0,691274020,39963,KD,"Kyndryl Holdings, Inc.",United States,IT Services,0.699,0.184,-0.596,0.796,1.271,1.528,2021,KYNDRYL HOLDINGS INC,4.835787e+09
482264,03/18/2022,TSV IT services,Software & Services,SAM GICS,Information Technology,Dimension,Social Dimension,Social Dimension,18.0,27.0,101.0,691274020,39963,KD,"Kyndryl Holdings, Inc.",United States,IT Services,0.699,0.184,-0.596,0.796,1.271,1.528,2021,KYNDRYL HOLDINGS INC,4.835787e+09
482265,03/18/2022,TSV IT services,Software & Services,SAM GICS,Information Technology,Dimension,Environmental Dimension,Environmental Dimension,22.0,21.0,91.0,691274020,39963,KD,"Kyndryl Holdings, Inc.",United States,IT Services,0.699,0.184,-0.596,0.796,1.271,1.528,2021,KYNDRYL HOLDINGS INC,4.835787e+09
482266,03/18/2022,TSV IT services,Software & Services,SAM GICS,Information Technology,Dimension,Economic Governance Dimension,Economic Governance Dimension,38.0,52.0,50.0,691274020,39963,KD,"Kyndryl Holdings, Inc.",United States,IT Services,0.699,0.184,-0.596,0.796,1.271,1.528,2021,KYNDRYL HOLDINGS INC,4.835787e+09


In [30]:
print(combined_df.companyname.unique().size, combined_df.gvkey.unique().size, combined_df.ticker.unique().size)

2221 2221 2221


In [31]:
combined_df.simpleindustry.unique()

array(['Banks', 'Capital Markets', 'Thrifts and Mortgage Finance',
       'IT Services', 'Hotels, Restaurants and Leisure',
       'Consumer Finance', 'Insurance',
       'Health Care Providers and Services',
       'Commercial Services and Supplies',
       'Trading Companies and Distributors', 'Industrial Conglomerates',
       'Technology Hardware, Storage and Peripherals', 'Road and Rail',
       'Electronic Equipment, Instruments and Components', 'Software',
       'Real Estate Management and Development', 'Automobiles',
       'Professional Services', 'Machinery', 'Electric Utilities',
       'Containers and Packaging', 'Internet and Direct Marketing Retail',
       'Aerospace and Defense', 'Textiles, Apparel and Luxury Goods',
       'Oil, Gas and Consumable Fuels', 'Food and Staples Retailing',
       'Electrical Equipment', 'Pharmaceuticals', 'Auto Components',
       'Food Products', 'Household Products', 'Household Durables',
       'Diversified Consumer Services', 'Multi-Ut

In [32]:
combined_df.csaindustrygroupname.unique()

array(['Banks', 'Diversified Financials', 'Software & Services',
       'Consumer Services', 'Insurance',
       'Health Care Equipment & Services',
       'Commercial & Professional Services', 'Capital Goods',
       'Technology Hardware & Equipment', 'Transportation', 'Real Estate',
       'Automobiles & Components', 'Utilities', 'Materials', 'Retailing',
       'Consumer Durables & Apparel', 'Energy',
       'Food & Staples Retailing',
       'Pharmaceuticals, Biotechnology & Life Sciences',
       'Food, Beverage & Tobacco', 'Household & Personal Products',
       'Media & Entertainment', 'Telecommunication Services',
       'Semiconductors & Semiconductor Equipment'], dtype=object)

In [33]:
#subsetting to dimensional datasets
environmental = combined_df.loc[combined_df['dimensionname'] == 'Environmental Dimension' ]
social = combined_df.loc[combined_df['dimensionname'] == 'Social Dimension' ]
governance = combined_df.loc[combined_df['dimensionname'] == 'Economic Governance Dimension']

In [34]:
environmental.to_csv('environmental_data.csv')
social.to_csv('social_data.csv')
governance.to_csv('governance_data.csv')